<a href="https://colab.research.google.com/github/zhong338/pingzhong/blob/main/Multifactor_Model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set up the data 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']= ['SimHei']

import datetime
import statsmodels.api as sm 
from scipy import stats

### Clarification：
1. df_all :store all stock information, including stock code, turnover rate, growth rate, PE, and PB
2. di_stock :Store stock basic information, including stock code, name, industry code, industry name
3. df_info : Store Monthly turnover and outstanding share capital of deposited stock, and use to calculate turnover rate
4. date: Store the entire month interval

In [ ]:
df_all = pd. read_excel(r'D:\data\月度数据_1.2.0.xlsx',header = 1,index_col = 0)
df_stock = pd.read_excel(r'D:\data\A股成分.xlsx',header = 1)
date = df_all.columns

In [ ]:
df_all[:3]

,2012-01-31,2012-02-29,2012-03-30,2012-04-27,2012-05-31,2012-06-29,2012-07-31,2012-08-31,2012-09-28,2012-10-31,...,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29,2019-12-31
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,1.162892,1.195039,1.097898,1.104502,1.046476,1.011127,0.969181,0.910774,0.842739,0.824993,...,1.000162,1.046423,0.920248,1.041135,1.041614,1.043825,1.128369,1.176862,1.106656,1.169181
000002.SZ,1.588009,1.718787,1.718787,1.816410,1.871024,1.802251,1.841854,1.600397,1.682213,1.624242,...,2.177156,2.071742,1.915357,1.994984,2.018730,1.810961,1.749978,1.792545,1.871598,1.933989
000004.SZ,9.218794,10.511878,9.062732,9.507560,10.466119,9.440684,8.989553,8.944492,9.147264,9.225603,...,16.659146,18.526989,15.756735,15.551531,14.476452,15.525016,15.899769,15.383872,16.838532,1.364990


In [ ]:
df_stock[:3]

,Code,名称,行业类别,行业代码
0,000001.SZ,平安银行,银行,S480000
1,000002.SZ,万科A,房地产,S430000
2,000004.SZ,国华网安,医药生物,S370000


In [ ]:
#一个月换手率数据
Turnover = pd.read_excel(r'D:\data\月度数据_1.2.0.xlsx', header = 1,index_col = 0, sheet_name = '换手率')
Turnover[:3]

,2012-01-31,2012-02-29,2012-03-30,2012-04-27,2012-05-31,2012-06-29,2012-07-31,2012-08-31,2012-09-28,2012-10-31,...,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29,2019-12-31
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,5.9227,15.1235,16.6545,11.3631,11.4935,12.8000,8.8032,9.2906,9.7387,6.9194,...,15.5577,15.1028,10.8616,9.6805,9.6233,17.4331,13.7180,10.4053,8.4517,7.8140
000002.SZ,10.1242,17.4246,12.0537,8.9494,11.4631,10.8400,13.0062,12.0015,10.5298,8.1784,...,16.3671,13.2516,6.7084,6.4802,9.8128,8.9136,9.9530,11.9469,12.2058,19.4650
000004.SZ,29.6126,54.1288,49.7603,27.0772,32.1800,49.6159,207.0672,56.9107,41.9219,22.2088,...,26.6422,37.6220,34.4733,26.0555,22.0360,19.7063,36.3557,13.3637,27.4346,37.2601


In [ ]:
#成交量数据 & 流通股本数据
Volume = pd.read_excel(r'D:\data\月度数据_1.2.0.xlsx', sheet_name = '成交量', header =1, index_col = 0)
CapitalStock = pd.read_excel(r'D:\data\月度数据_1.2.0.xlsx', sheet_name = '流通股本', header =1, index_col = 0)
Volume[:3]

,2012-01-31,2012-02-29,2012-03-30,2012-04-27,2012-05-31,2012-06-29,2012-07-31,2012-08-31,2012-09-28,2012-10-31,...,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29,2019-12-31
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,183921042.0,4.696358e+08,5.171779e+08,352860001.0,3.569099e+08,3.974864e+08,2.733712e+08,2.885086e+08,3.024194e+08,214870356.0,...,2.671303e+09,2.593170e+09,1.864961e+09,1.662158e+09,1.652336e+09,2.993290e+09,2.538655e+09,2.019234e+09,1.640143e+09,1.516355e+09
000002.SZ,978179964.0,1.683528e+09,1.164602e+09,864667366.0,1.107535e+09,1.047364e+09,1.256639e+09,1.159596e+09,1.017391e+09,790201889.0,...,1.590086e+09,1.287407e+09,6.517398e+08,6.295582e+08,9.533243e+08,8.659557e+08,9.669779e+08,1.160658e+09,1.185824e+09,1.891029e+09
000004.SZ,24837662.0,4.540058e+07,4.173676e+07,22711235.0,2.699105e+07,4.161562e+07,1.736786e+08,4.773399e+07,3.516226e+07,18627774.0,...,2.209531e+07,3.120119e+07,2.858977e+07,2.160858e+07,1.820276e+07,1.627824e+07,3.003140e+07,1.103910e+07,2.266200e+07,3.101480e+07


In [ ]:
CapitalStock[:3]

,2012-01-31,2012-02-29,2012-03-30,2012-04-27,2012-05-31,2012-06-29,2012-07-31,2012-08-31,2012-09-28,2012-10-31,...,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29,2019-12-31
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,3.105359e+09,3.105359e+09,3.105359e+09,3.105359e+09,3.105359e+09,3.105359e+09,3.105359e+09,3.105359e+09,3.105359e+09,3.105359e+09,...,1.717025e+10,1.717025e+10,1.717025e+10,1.717025e+10,1.717025e+10,1.717025e+10,1.940575e+10,1.940575e+10,1.940575e+10,1.940575e+10
000002.SZ,9.661828e+09,9.661828e+09,9.661828e+09,9.661828e+09,9.661828e+09,9.661828e+09,9.662036e+09,9.662036e+09,9.662036e+09,9.662036e+09,...,9.715170e+09,9.715170e+09,9.715170e+09,9.715170e+09,9.715170e+09,9.715170e+09,9.715170e+09,9.715170e+09,9.715170e+09,9.715170e+09
000004.SZ,8.387546e+07,8.387546e+07,8.387546e+07,8.387546e+07,8.387546e+07,8.387546e+07,8.387546e+07,8.387546e+07,8.387546e+07,8.387546e+07,...,8.293361e+07,8.293361e+07,8.293361e+07,8.293361e+07,8.260458e+07,8.260458e+07,8.260458e+07,8.260458e+07,8.260458e+07,8.392068e+07


In [ ]:
#准备三个月换手率 sto_3m
sto_3m = pd.DataFrame()

for i in range (2,len(date)):
    sto_3m[date[i]] = (Volume[date[i]]+Volume[date[i-1]]
                                 +Volume[date[i-2]])/CapitalStock[date[i]]
sto_3m[:3]

,2012-03-30,2012-04-27,2012-05-31,2012-06-29,2012-07-31,2012-08-31,2012-09-28,2012-10-31,2012-11-30,2012-12-31,...,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29,2019-12-31
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,0.377005,0.431407,0.395107,0.356563,0.330966,0.308939,0.278325,0.259486,0.240414,0.368488,...,0.372685,0.417929,0.415220,0.356447,0.301653,0.367367,0.370214,0.389121,0.319391,0.266711
000002.SZ,0.396023,0.384275,0.324660,0.312525,0.353087,0.358475,0.355373,0.307098,0.285657,0.338312,...,0.341157,0.402909,0.363270,0.264401,0.230014,0.252063,0.286795,0.308136,0.341060,0.436175
000004.SZ,1.335015,1.309663,1.090176,1.088732,2.888631,3.135938,3.058998,1.210414,0.822744,0.627093,...,0.468050,0.777644,0.987371,0.981502,0.828055,0.679013,0.780978,0.694256,0.771537,0.771156


In [ ]:
#准备六个月换手率 sto_6m
sto_6m = pd.DataFrame()

for i in range (5,len(date)):
    k=5
    s=0
    while k>=0:
        s=s+Volume[date[i-k]]
        k=k-1
    sto_6m[date[i]]=s/Volume[date[i]]
sto_6m[:3]

,2012-06-29,2012-07-31,2012-08-31,2012-09-28,2012-10-31,2012-11-30,2012-12-31,2013-01-31,2013-02-28,2013-03-29,...,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29,2019-12-31
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,5.730991,8.660170,7.577985,6.519275,8.533359,7.440371,2.868855,2.333910,3.807818,3.027048,...,4.665718,4.734191,6.464776,7.532003,7.477536,4.489113,5.240795,6.304686,7.624834,8.151134
000002.SZ,6.536293,5.669358,5.691984,6.342885,8.072275,6.534423,4.391612,4.688463,4.587558,4.017847,...,3.899935,4.937939,9.296884,9.344808,6.450008,6.903437,5.537834,4.504525,4.859320,3.714258
000004.SZ,4.885014,2.027503,7.425887,9.893925,18.456811,21.818607,16.487548,3.311956,5.042040,6.024668,...,3.163289,3.121810,3.801793,5.563375,7.300758,8.476090,4.858646,11.391311,5.287357,4.166666


In [ ]:
#涨跌幅数据
ret = pd.read_excel(r'D:\data\月度数据_1.2.0.xlsx', sheet_name = '涨跌幅', header =1, index_col = 0)
ret

,2012-01-31,2012-02-29,2012-03-30,2012-04-27,2012-05-31,2012-06-29,2012-07-31,2012-08-31,2012-09-28,2012-10-31,...,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29,2019-12-31
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,6.735087,2.764423,-8.128655,5.410567,-5.253623,-3.377948,-0.395778,-6.026490,-7.470049,0.975900,...,3.721683,8.034321,-12.057762,14.328093,2.539913,0.212314,10.098870,4.297627,-5.965560,7.586658
000002.SZ,2.409639,8.235294,0.000000,8.454106,3.006682,-3.675676,5.091124,-13.109426,5.112219,-1.304864,...,9.753483,-5.989583,-7.548476,4.157303,3.416037,-6.680365,0.387597,2.432432,4.410102,16.173285
000004.SZ,2.224969,14.026602,-13.785790,4.920049,10.082063,-9.797657,-5.785124,-0.501253,2.267003,1.354680,...,11.413882,16.658976,-14.952532,-1.302326,-12.818096,7.243243,-5.241935,-3.244681,9.455745,13.460573
000005.SZ,NaN,NaN,NaN,2.849741,-11.083123,-14.730878,-9.302326,1.831502,2.517986,1.754386,...,9.593023,-12.466844,-8.181818,0.330033,-3.947368,6.164384,2.580645,-5.031447,-2.980132,5.460751
000006.SZ,7.073171,20.045558,4.554080,15.245009,-3.622047,-6.699346,-6.064823,-2.195122,0.000000,3.491272,...,10.655738,-9.629630,-10.000000,2.751396,-2.935780,0.189036,-5.094340,3.976143,-2.103250,4.687500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688699.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688777.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688788.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def mad(x,n):
    dm=x.median()
    dmad=((x-dm).abs()).median()
    max=dm+n*dmad
    min=dm-n*dmad
    return np.clip(x,min,max)

In [ ]:
sto_1m_s = pd.DataFrame()
for i in range(0,len(date)):
    sto_1m_s[date[i]]=mad(Turnover[date[i]],5)

In [ ]:
sto_3m_s = pd.DataFrame()
for i in range(2,len(date)):
    sto_3m_s[date[i]]=mad(sto_3m[date[i]],5)
sto_6m_s = pd.DataFrame()
for i in range(5,len(date)):
    sto_6m_s[date[i]]=mad(sto_6m[date[i]],5)

In [ ]:
ret_s = pd.DataFrame()
for i in range(0,len(date)):
    ret_s[date[i]]=mad(ret[date[i]],5)

In [ ]:
def standard(x):
    sigma=x.std()
    u=x.mean()
    return (x-u)/sigma

In [ ]:
for i in range(0,len(date)):
    sto_1m_s[date[i]]=standard(Turnover[date[i]])
    
for i in range(2,len(date)):
    sto_3m_s[date[i]]=standard(sto_3m_s[date[i]])
    
for i in range(5,len(date)):
    sto_6m_s[date[i]]=standard(sto_6m_s[date[i]])
    
for i in range(0,len(date)):
    ret_s[date[i]]=standard(ret_s[date[i]])

In [ ]:
df_stock[:3]

,Code,名称,行业类别,行业代码
0,000001.SZ,平安银行,银行,S480000
1,000002.SZ,万科A,房地产,S430000
2,000004.SZ,国华网安,医药生物,S370000


In [ ]:
test = pd.DataFrame(ret_s[date[1]])
test.columns = ['涨跌幅'] 

In [ ]:
test['换手率']=sto_1m_s[date[0]]
test['行业类别']= df_stock['行业类别'].values
test[:3]

,涨跌幅,换手率,行业类别
Code,,,
000001.SZ,-1.503265,-0.579758,银行
000002.SZ,-0.700581,-0.467909,房地产
000004.SZ,0.149118,0.050895,医药生物


In [ ]:
test= test.dropna()
test[:3]

,涨跌幅,换手率,行业类别
Code,,,
000001.SZ,-1.503265,-0.579758,银行
000002.SZ,-0.700581,-0.467909,房地产
000004.SZ,0.149118,0.050895,医药生物


(3) 行业因子处理
第T期，第J个行业因子；
one-hot编码

In [ ]:
test =pd.get_dummies(test)
test[:3]

,涨跌幅,换手率,行业类别_交通运输,行业类别_休闲服务,行业类别_传媒,行业类别_公用事业,行业类别_农林牧渔,行业类别_化工,行业类别_医药生物,行业类别_商业贸易,...,行业类别_纺织服装,行业类别_综合,行业类别_计算机,行业类别_轻工制造,行业类别_通信,行业类别_采掘,行业类别_钢铁,行业类别_银行,行业类别_非银金融,行业类别_食品饮料
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,-1.503265,-0.579758,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
000002.SZ,-0.700581,-0.467909,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000004.SZ,0.149118,0.050895,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


(4) 回归分析

In [ ]:
Y=test['涨跌幅']
X= test[test.columns[1:]]
est=sm.OLS(Y,sm.add_constant(X)).fit()
print('第一个截面的回归系数:',est.params['换手率'])
print('第一个截面的T值:',est.tvalues['换手率'])

第一个截面的回归系数: 0.14847375113454653
第一个截面的T值: 7.282280655485534


In [ ]:
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    涨跌幅   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.096
Method:                 Least Squares   F-statistic:                     9.419
Date:                Sat, 30 Jan 2021   Prob (F-statistic):           1.36e-37
Time:                        20:00:45   Log-Likelihood:                -2982.7
No. Observations:                2218   AIC:                             6023.
Df Residuals:                    2189   BIC:                             6189.
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0654      0.023     -2.855      0.004      -0.110      -0.020
换手率            0.1485      0.020      7.282      0.000       0.108       0.188
行业类别_交通运输     -0.3253      0.103     -3.158      0.002      -0.527      -0.123
行业类别_休闲服务     -0.1424      0.172     -0.828      0.408      -0.480       0.195
行业类别_传媒        0.2351      0.093      2.517      0.012       0.052       0.418
行业类别_公用事业     -0.2029      0.089     -2.290      0.022      -0.377      -0.029
行业类别_农林牧渔     -0.0493      0.119     -0.415      0.678      -0.282       0.183
行业类别_化工        0.0408      0.069      0.593      0.554      -0.094       0.176
行业类别_医药生物      0.0258      0.070      0.367      0.713      -0.112       0.164
行业类别_商业贸易     -0.2921      0.103     -2.823      0.005      -0.495      -0.089
行业类别_国防军工      0.0688      0.132      0.521      0.603      -0.190       0.328
行业类别_家用电器      0.4394      0.154      2.850      0.004       0.137       0.742
行业类别_建筑材料      0.0243      0.131      0.185      0.853      -0.233       0.281
行业类别_建筑装饰      0.1104      0.119      0.930      0.352      -0.122       0.343
行业类别_房地产       0.5728      0.085      6.716      0.000       0.406       0.740
行业类别_有色金属     -0.3988      0.106     -3.757      0.000      -0.607      -0.191
行业类别_机械设备      0.0340      0.076      0.445      0.656      -0.116       0.183
行业类别_汽车        0.1448      0.099      1.464      0.143      -0.049       0.339
行业类别_电子        0.3830      0.086      4.458      0.000       0.215       0.551
行业类别_电气设备      0.0332      0.086      0.387      0.699      -0.135       0.202
行业类别_纺织服装      0.1445      0.134      1.082      0.279      -0.117       0.406
行业类别_综合       -0.0234      0.182     -0.128      0.898      -0.380       0.333
行业类别_计算机       0.3725      0.088      4.210      0.000       0.199       0.546
行业类别_轻工制造      0.2797      0.117      2.395      0.017       0.051       0.509
行业类别_通信        0.4705      0.128      3.665      0.000       0.219       0.722
行业类别_采掘       -0.2592      0.127     -2.038      0.042      -0.509      -0.010
行业类别_钢铁       -0.3720      0.166     -2.234      0.026      -0.698      -0.045
行业类别_银行       -1.3805      0.227     -6.085      0.000      -1.825      -0.936
行业类别_非银金融     -0.0973      0.123     -0.793      0.428      -0.338       0.143
行业类别_食品饮料      0.0981      0.120      0.820      0.412      -0.137       0.333
==============================================================================
Omnibus:                      196.293   Durbin-Watson:                   1.934
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              262.195
Skew:                           0.734   Prob(JB):                     1.16e-57
Kurtosis:                       3.825   Cond. No.                     1.82e+16
==============================================================================

W

In [ ]:
df_stock[:3]

,Code,名称,行业类别,行业代码
0,000001.SZ,平安银行,银行,S480000
1,000002.SZ,万科A,房地产,S430000
2,000004.SZ,国华网安,医药生物,S370000


In [ ]:
def find_tvalue(x,index):
    test=pd.DataFrame(ret[date[x+1]][:])
    test.columns = ['涨跌幅']
    
    test['换手率']=Turnover[date[x]][:]
    test['行业类别']= df_stock['行业类别'].values
    
    #去极值
    test['涨跌幅']= mad(test['涨跌幅'],5)
    test['换手率']= mad(test['换手率'],5)
    
    #标准化
    test['涨跌幅']=standard(test['涨跌幅'])
    test['换手率']=standard(test['换手率'])
    
    #去缺失值
    test= test.dropna()
    
    #行业dummy
    test=pd.get_dummies(test)
    
    #回归
    Y=test['涨跌幅']
    X=test[test.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    return est.tvalues[index],est.params[index]
   

In [ ]:
find_tvalue(0,'换手率')[0]

8.31799824567405

In [ ]:
index = '换手率'
result_t = []
result_r=[]
for i in range(len(date)-1):
    result_t.append(find_tvalue(i,index)[0])
    result_r.append(find_tvalue(i,index)[1])

In [ ]:
result_t[:3]

[8.31799824567405, -11.922266036340494, -8.540815296768027]

In [ ]:
score1=np.mean([abs(x) for x in result_t])
print('T值序列的绝对值平均值:',score1)

T值序列的绝对值平均值: 7.672359328573732


In [ ]:
score2=np.mean([abs(x)>2 for x in result_t])
print('T值序列的绝对值大于2的比例:',score2)

T值序列的绝对值大于2的比例: 0.8631578947368421


In [ ]:
score3=stats.ttest_1samp(result_r,0)[0]
print('收益率序列的T值',score3)

收益率序列的T值 -4.914079572347546


In [ ]:
def score(index):
    result_t=[]
    result_r=[]
    for i in range(len(date)-1):
        result_t.append(find_tvalue(i,index)[0])
        result_r.append(find_tvalue(i,index)[1])
    score1=np.mean([abs(x) for x in result_t])
    print('T值序列的绝对值平均值:',score1)
    score2=np.mean([abs(x)>2 for x in result_t])
    print('T值序列的绝对值大于2的比例:',score2)
    score3=stats.ttest_1samp(result_r,0)[0]
    print('收益率序列的T值',score3)
    return score1, score2, score3

In [ ]:
score('换手率')

T值序列的绝对值平均值: 7.672359328573732
T值序列的绝对值大于2的比例: 0.8631578947368421
收益率序列的T值 -4.914079572347546


(7.672359328573732, 0.8631578947368421, -4.914079572347546)

In [ ]:
data1=pd.DataFrame(index=df_stock['Code'])
for i in date[11:]:
    s = sto_1m_s[i]+sto_3m_s[i]+sto_6m_s[i]
    data1[i]=s*(1/3)

In [ ]:
data1.dropna()[:3]

,2012-12-31,2013-01-31,2013-02-28,2013-03-29,2013-04-26,2013-05-31,2013-06-28,2013-07-31,2013-08-30,2013-09-30,...,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29,2019-12-31
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,-0.780419,-0.750598,-0.561355,-0.386759,-0.217589,-0.006530,-0.433339,-0.475630,-0.326091,-0.341030,...,-0.147866,-0.638581,-0.762482,-0.692755,-0.665061,-0.813423,-0.697781,-0.656704,-0.533541,-0.243218
000011.SZ,0.745844,0.788569,0.984776,0.836420,0.716773,0.876917,0.996666,0.900382,0.946922,1.028758,...,0.538931,0.713522,0.756161,0.754623,-0.561748,-0.600712,-0.467903,0.100578,0.184240,-0.024043
000012.SZ,-0.584792,-0.323824,-0.349945,-0.436794,-0.404552,-0.372557,-0.034022,0.080967,0.186975,0.232804,...,-0.660512,-0.250615,-0.177370,-0.351333,-0.246616,-0.605900,-0.773886,-0.619553,-0.446775,-0.792061


In [ ]:
data2 = pd.DataFrame()

for i in range(11, len(date)-1):
    d0= date[i]
    d1= date[i+1]
    
    df1= pd.DataFrame(ret[d1][:])
    df1.columns=['涨跌幅']
    df1['换手率1']=sto_1m_s[d0]
    df1['换手率3']=sto_3m_s[d0]
    df1['换手率6']=sto_6m_s[d0]
    
    df1= df1.dropna()
    
    Y=df1['涨跌幅']
    X=df1[df1.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    result = np.array(est.params[['换手率1','换手率3','换手率6']])
    ratio = abs(result)/sum(abs(result))
    
    data2[d0]=sto_1m_s[d0]*ratio[0]+sto_3m_s[d0]*ratio[0]+sto_6m_s[d0]*ratio[2]

In [ ]:
data2.dropna()[:3]

,2012-12-31,2013-01-31,2013-02-28,2013-03-29,2013-04-26,2013-05-31,2013-06-28,2013-07-31,2013-08-30,2013-09-30,...,2019-02-28,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,-1.146517,-0.555829,-0.305103,-0.029655,0.016060,-0.143952,-0.276605,-0.365883,-0.140047,-0.239373,...,-0.875021,-1.089762,-0.158859,-1.566108,-1.241731,-0.358276,-0.842104,-1.123460,-0.767867,-0.457064
000011.SZ,1.144727,0.049131,0.541326,0.265217,1.026660,1.596872,1.056297,0.666301,0.415319,1.244872,...,0.943191,1.193062,1.090682,0.520711,0.051580,-0.532263,-0.619407,-0.791177,-0.119477,-0.165850
000012.SZ,-0.854543,-0.085091,-0.201771,-0.585640,-0.352295,-0.094204,0.259090,0.049541,0.082810,0.117575,...,-0.875146,-0.984028,0.548610,-1.226581,-1.073488,0.335622,-0.794993,-1.127546,-0.743005,-0.424175


In [ ]:
sto_3m_s

,2012-03-30,2012-04-27,2012-05-31,2012-06-29,2012-07-31,2012-08-31,2012-09-28,2012-10-31,2012-11-30,2012-12-31,...,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29,2019-12-31
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,-0.909864,-0.909982,-0.929380,-0.874069,-0.864083,-0.830541,-0.876547,-0.890369,-0.902221,-0.762624,...,-1.017731,-1.094692,-1.081264,-0.990686,-0.914127,-0.821260,-0.842208,-0.804470,-0.852744,-0.902624
000002.SZ,-0.888711,-0.958271,-1.008307,-0.931281,-0.835866,-0.766951,-0.783663,-0.828736,-0.834630,-0.804595,...,-1.043343,-1.105393,-1.118128,-1.071311,-0.989212,-0.941966,-0.920795,-0.884424,-0.830096,-0.715554
000004.SZ,0.155624,-0.010185,-0.150642,0.077131,2.147898,2.080903,2.135432,0.340623,-0.032243,-0.402931,...,-0.940258,-0.838428,-0.675261,-0.443187,-0.362409,-0.495014,-0.455236,-0.503219,-0.380175,-0.345769
000005.SZ,NaN,NaN,NaN,-0.620429,-0.641348,-0.559896,-0.645634,-0.676749,-0.754717,-0.608015,...,-0.349270,-0.434396,-0.506434,-0.572626,-0.512969,-0.379960,-0.050090,-0.105736,-0.200722,-0.714839
000006.SZ,-0.708500,-0.303364,-0.167363,-0.067272,-0.699574,-0.734513,-0.811701,-0.874102,-0.785555,-0.443436,...,-0.552053,-0.602669,-0.677357,-0.857348,-0.896014,-0.792686,-0.716926,-0.724987,-0.832805,-0.876891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688699.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688777.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688788.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


计算市净率PB： df_all

In [ ]:
def find_tvalue(x,index):
    test=pd.DataFrame(ret[date[x+1]][:])
    test.columns = ['涨跌幅']
    
    test['PB']=df_all[date[x]][:]
    test['行业类别']= df_stock['行业类别'].values
    
    #去极值
    test['涨跌幅']= mad(test['涨跌幅'],5)
    test['PB']= mad(test['PB'],5)
    
    #标准化
    test['涨跌幅']=standard(test['涨跌幅'])
    test['PB']=standard(test['PB'])
    
    #去缺失值
    test= test.dropna()
    
    #行业dummy
    test=pd.get_dummies(test)
    
    #回归
    Y=test['涨跌幅']
    X=test[test.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    return est.tvalues[index],est.params[index]
   

In [ ]:
find_tvalue(0,'PB')[0]

0.14479307769837632

In [ ]:
index = 'PB'
result_t = []
result_r=[]
for i in range(len(date)-1):
    result_t.append(find_tvalue(i,index)[0])
    result_r.append(find_tvalue(i,index)[1])

In [ ]:
result_t[:3]

[0.14479307769837632, 0.8739382449336776, -4.479322390647813]

In [ ]:
score('PB')

T值序列的绝对值平均值: 4.2286978829379525
T值序列的绝对值大于2的比例: 0.6736842105263158
收益率序列的T值 -4.692481087809354


(4.2286978829379525, 0.6736842105263158, -4.692481087809354)

In [ ]:
data_PB = pd.DataFrame()

for i in range(11, len(date)-1):
    d0= date[i]
    d1= date[i+1]
    
    df1= pd.DataFrame(ret[d1][:])
    df1.columns=['涨跌幅']
    df1['PB']=df_all[d0]
    
    df1= df1.dropna()
    
    Y=df1['涨跌幅']
    X=df1[df1.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    result = np.array(est.params[['PB']])
    ratio = abs(result)/sum(abs(result))
    
    data_PB[d0]=df_all[d0]*ratio[0]

In [ ]:
data_PB.dropna()[:3]

,2012-12-31,2013-01-31,2013-02-28,2013-03-29,2013-04-26,2013-05-31,2013-06-28,2013-07-31,2013-08-30,2013-09-30,...,2019-02-28,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,0.967892,1.274814,1.389608,1.215605,1.079402,1.211586,0.920782,0.858761,0.950099,1.026041,...,0.964275,1.000162,1.046423,0.920248,1.041135,1.041614,1.043825,1.128369,1.176862,1.106656
000002.SZ,1.743424,2.069024,2.067301,1.853680,1.851300,2.010902,1.653371,1.573280,1.563364,1.473000,...,1.983678,2.177156,2.071742,1.915357,1.994984,2.018730,1.810961,1.749978,1.792545,1.871598
000004.SZ,8.868886,10.104892,10.896369,10.343419,11.416526,12.586076,10.729818,10.931607,13.174266,13.645002,...,14.952487,16.659146,18.526989,15.756735,15.551531,14.476452,15.525016,15.899769,15.383872,16.838532


计算市赢率PE：

In [ ]:
PE= pd.read_excel(r'D:\data\月度数据_1.2.0.xlsx', sheet_name = '市盈率', header =1, index_col = 0)
PE[:3]

,2012-01-31,2012-02-29,2012-03-30,2012-04-27,2012-05-31,2012-06-29,2012-07-31,2012-08-31,2012-09-28,2012-10-31,...,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29,2019-12-31
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,8.294154,8.523440,7.830599,8.254279,7.820630,7.556453,7.526546,7.072960,6.544606,6.559559,...,8.869557,9.582166,8.426771,9.533736,9.775885,9.796641,12.190276,12.714168,11.955697,11.322126
000002.SZ,8.739163,9.458859,9.458859,10.258521,10.566962,10.178555,10.544115,9.161842,9.630215,9.504554,...,10.041342,9.664799,8.935254,9.306720,9.624641,8.634066,8.667531,8.878363,9.269908,9.356404
000004.SZ,232.278423,264.859193,228.346261,239.581009,263.735718,237.895797,224.133230,223.009755,228.065392,231.154948,...,-89.773280,-104.728588,-89.069013,-87.909045,-76.640779,-82.192057,-77.883602,-75.356528,-82.482049,612.071032


In [ ]:
def find_tvalue(x,index):
    test=pd.DataFrame(ret[date[x+1]][:])
    test.columns = ['涨跌幅']
    
    test['PE']=PE[date[x]][:]
    test['行业类别']= df_stock['行业类别'].values
    
    #去极值
    test['涨跌幅']= mad(test['涨跌幅'],5)
    test['PE']= mad(test['PE'],5)
    
    #标准化
    test['涨跌幅']=standard(test['涨跌幅'])
    test['PE']=standard(test['PE'])
    
    #去缺失值
    test= test.dropna()
    
    #行业dummy
    test=pd.get_dummies(test)
    
    #回归
    Y=test['涨跌幅']
    X=test[test.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    return est.tvalues[index],est.params[index]

In [ ]:
find_tvalue(0,'PE')[0]

0.14479307769837632

In [ ]:
index = 'PE'
result_t = []
result_r=[]
for i in range(len(date)-1):
    result_t.append(find_tvalue(i,index)[0])
    result_r.append(find_tvalue(i,index)[1])
result_t[:3]

[0.14479307769837632, 0.8739382449336776, -4.479322390647813]

In [ ]:
score('PE')

T值序列的绝对值平均值: 4.2286978829379525
T值序列的绝对值大于2的比例: 0.6736842105263158
收益率序列的T值 -4.692481087809354


(4.2286978829379525, 0.6736842105263158, -4.692481087809354)

In [ ]:
data_PE = pd.DataFrame()
for i in range(11, len(date)-1):
    d0= date[i]
    d1= date[i+1]
    
    df1= pd.DataFrame(ret[d1][:])
    df1.columns=['涨跌幅']
    df1['PE']=PE[d0]
    
    df1= df1.dropna()
    
    Y=df1['涨跌幅']
    X=df1[df1.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    result = np.array(est.params[['PE']])
    ratio = abs(result)/sum(abs(result))
    
    data_PE[d0]=PE[d0]*ratio[0]
    
data_PE.dropna()[:3]

,2012-12-31,2013-01-31,2013-02-28,2013-03-29,2013-04-26,2013-05-31,2013-06-28,2013-07-31,2013-08-30,2013-09-30,...,2019-02-28,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,0.967892,1.274814,1.389608,1.215605,1.079402,1.211586,0.920782,0.858761,0.950099,1.026041,...,0.964275,1.000162,1.046423,0.920248,1.041135,1.041614,1.043825,1.128369,1.176862,1.106656
000002.SZ,1.743424,2.069024,2.067301,1.853680,1.851300,2.010902,1.653371,1.573280,1.563364,1.473000,...,1.983678,2.177156,2.071742,1.915357,1.994984,2.018730,1.810961,1.749978,1.792545,1.871598
000004.SZ,8.868886,10.104892,10.896369,10.343419,11.416526,12.586076,10.729818,10.931607,13.174266,13.645002,...,14.952487,16.659146,18.526989,15.756735,15.551531,14.476452,15.525016,15.899769,15.383872,16.838532


In [ ]:
Yield= pd.read_excel(r'D:\data\月度数据_1.2.0.xlsx', sheet_name = '股息率', header =1, index_col = 0)
Yield[:3]

,2012-01-31,2012-02-29,2012-03-30,2012-04-27,2012-05-31,2012-06-29,2012-07-31,2012-08-31,2012-09-28,2012-10-31,...,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29,2019-12-31
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,0.017868,0.017388,0.018926,0.017955,0.018950,0.019613,0.019691,0.020953,0.022645,0.022593,...,0.006920,0.006406,0.007284,0.006438,0.006279,0.006265,0.005035,0.004828,0.005134,0.004772
000002.SZ,0.053691,0.049606,0.049606,0.045739,0.044404,0.046098,0.044500,0.051214,0.048723,0.049367,...,0.013317,0.013836,0.014966,0.014368,0.013894,0.015488,0.015428,0.015062,0.014425,0.012417
000004.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def find_tvalue(x,index):
    test=pd.DataFrame(ret[date[x+1]][:])
    test.columns = ['涨跌幅']
    
    test['Yield']=Yield[date[x]][:]
    test['行业类别']= df_stock['行业类别'].values
    
    #去极值
    test['涨跌幅']= mad(test['涨跌幅'],5)
    test['Yield']= mad(test['Yield'],5)
    
    #标准化
    test['涨跌幅']=standard(test['涨跌幅'])
    test['Yield']=standard(test['Yield'])
    
    #去缺失值
    test= test.dropna()
    
    #行业dummy
    test=pd.get_dummies(test)
    
    #回归
    Y=test['涨跌幅']
    X=test[test.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    return est.tvalues[index],est.params[index]
find_tvalue(0,'Yield')[0]

-2.6850709047603543

In [ ]:
index = 'Yield'
result_t = []
result_r=[]
for i in range(len(date)-1):
    result_t.append(find_tvalue(i,index)[0])
    result_r.append(find_tvalue(i,index)[1])
result_t[:3]

[0.14479307769837632, 0.8739382449336776, -4.479322390647813]

In [ ]:
score('Yield')

T值序列的绝对值平均值: 4.2286978829379525
T值序列的绝对值大于2的比例: 0.6736842105263158
收益率序列的T值 -4.692481087809354


(4.2286978829379525, 0.6736842105263158, -4.692481087809354)

In [ ]:
data_Yield = pd.DataFrame()

for i in range(11, len(date)-1):
    d0= date[i]
    d1= date[i+1]
    
    df1= pd.DataFrame(ret[d1][:])
    df1.columns=['涨跌幅']
    df1['Yield']=Yield[d0]
    
    df1= df1.dropna()
    
    Y=df1['涨跌幅']
    X=df1[df1.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    result = np.array(est.params[['Yield']])
    ratio = abs(result)/sum(abs(result))
    
    data_Yield[d0]=Yield[d0]*ratio[0]
    
data_Yield.dropna()[:3]

,2012-12-31,2013-01-31,2013-02-28,2013-03-29,2013-04-26,2013-05-31,2013-06-28,2013-07-31,2013-08-30,2013-09-30,...,2019-02-28,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,0.018560,0.014091,0.012927,0.014778,0.015900,0.014165,0.018639,0.019378,0.017515,0.015642,...,0.007178,0.006920,0.006406,0.007284,0.006438,0.006279,0.006265,0.005035,0.004828,0.005134
000002.SZ,0.040585,0.034198,0.034227,0.038171,0.037199,0.034247,0.041652,0.043072,0.043345,0.044912,...,0.014616,0.013317,0.013836,0.014966,0.014368,0.013894,0.015488,0.015428,0.015062,0.014425
000006.SZ,0.032879,0.029430,0.031238,0.036167,0.036989,0.032550,0.036385,0.038366,0.032563,0.033550,...,0.025410,0.022963,0.025410,0.028233,0.028440,0.029301,0.029245,0.030815,0.029637,0.030273


In [ ]:
Total_value = pd.read_excel(r'D:\data\月度数据_1.2.0.xlsx', sheet_name = '总市值', header =1, index_col = 0)

In [ ]:
def find_tvalue(x,index):
    test=pd.DataFrame(ret[date[x+1]][:])
    test.columns = ['涨跌幅']
    
    test['总市值']=Total_value[date[x]][:]
    test['行业类别']= df_stock['行业类别'].values
    
    #去极值
    test['涨跌幅']= mad(test['涨跌幅'],5)
    test['总市值']= mad(test['总市值'],5)
    
    #标准化
    test['涨跌幅']=standard(test['涨跌幅'])
    test['总市值']=standard(test['总市值'])
    
    #去缺失值
    test= test.dropna()
    
    #行业dummy
    test=pd.get_dummies(test)
    
    #回归
    Y=test['涨跌幅']
    X=test[test.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    return est.tvalues[index],est.params[index]
find_tvalue(0,'总市值')[0]

-11.241221665785309

In [ ]:
index = '总市值'
result_t = []
result_r=[]
for i in range(len(date)-1):
    result_t.append(find_tvalue(i,index)[0])
    result_r.append(find_tvalue(i,index)[1])
result_t[:3]

[-11.241221665785309, 0.35071902639358377, -3.6407284630230583]

In [ ]:
score('总市值')

T值序列的绝对值平均值: 6.88705353201115
T值序列的绝对值大于2的比例: 0.8421052631578947
收益率序列的T值 -3.153804911469757


(6.88705353201115, 0.8421052631578947, -3.153804911469757)

In [ ]:
data_Total_value = pd.DataFrame()

for i in range(11, len(date)-1):
    d0= date[i]
    d1= date[i+1]
    
    df1= pd.DataFrame(ret[d1][:])
    df1.columns=['涨跌幅']
    df1['总市值']=Total_value[d0]
    
    df1= df1.dropna()
    
    Y=df1['涨跌幅']
    X=df1[df1.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    result = np.array(est.params[['总市值']])
    ratio = abs(result)/sum(abs(result))
    
    data_Total_value[d0]=Total_value[d0]*ratio[0]
    
data_Total_value.dropna()[:3]

,2012-12-31,2013-01-31,2013-02-28,2013-03-29,2013-04-26,2013-05-31,2013-06-28,2013-07-31,2013-08-30,2013-09-30,...,2019-02-28,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,8.207607e+10,1.081027e+11,1.178371e+11,1.030818e+11,9.580665e+10,1.075391e+11,8.172769e+10,7.861269e+10,8.697400e+10,9.738464e+10,...,2.122263e+11,2.201247e+11,2.378102e+11,2.091356e+11,2.366083e+11,2.426179e+11,2.431330e+11,3.025383e+11,3.155402e+11,2.967165e+11
000002.SZ,1.112750e+11,1.320566e+11,1.319466e+11,1.183122e+11,1.214045e+11,1.318709e+11,1.084248e+11,1.048507e+11,1.041898e+11,1.005553e+11,...,3.089859e+11,3.391227e+11,3.264059e+11,3.017672e+11,3.143126e+11,3.250496e+11,2.915953e+11,2.927255e+11,2.998459e+11,3.130694e+11
000004.SZ,6.869293e+08,7.826627e+08,8.439657e+08,8.011376e+08,8.935119e+08,9.850465e+08,8.397668e+08,8.473247e+08,1.021156e+09,1.069023e+09,...,1.633347e+09,1.819775e+09,2.122931e+09,1.805499e+09,1.781985e+09,1.553569e+09,1.666097e+09,1.578762e+09,1.527536e+09,1.671976e+09


In [ ]:
LTSZ = pd.read_excel(r'D:\data\月度数据_1.2.0.xlsx', sheet_name = '流通市值', header =1, index_col = 0)

In [ ]:
def find_tvalue(x,index):
    test=pd.DataFrame(ret[date[x+1]][:])
    test.columns = ['涨跌幅']
    
    test['流通市值']=LTSZ[date[x]][:]
    test['行业类别']= df_stock['行业类别'].values
    
    #去极值
    test['涨跌幅']= mad(test['涨跌幅'],5)
    test['流通市值']= mad(test['流通市值'],5)
    
    #标准化
    test['涨跌幅']=standard(test['涨跌幅'])
    test['流通市值']=standard(test['流通市值'])
    
    #去缺失值
    test= test.dropna()
    
    #行业dummy
    test=pd.get_dummies(test)
    
    #回归
    Y=test['涨跌幅']
    X=test[test.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    return est.tvalues[index],est.params[index]
find_tvalue(0,'流通市值')[0]

-11.719201175320997

In [ ]:
index = '流通市值'
result_t = []
result_r=[]
for i in range(len(date)-1):
    result_t.append(find_tvalue(i,index)[0])
    result_r.append(find_tvalue(i,index)[1])
result_t[:3]

[-11.719201175320997, -0.025269282332027694, 0.5863232012842595]

In [ ]:
score('流通市值')

T值序列的绝对值平均值: 6.912944804123067
T值序列的绝对值大于2的比例: 0.8
收益率序列的T值 -3.5802102271490197


(6.912944804123067, 0.8, -3.5802102271490197)

In [ ]:
data_LTSZ = pd.DataFrame()

for i in range(11, len(date)-1):
    d0= date[i]
    d1= date[i+1]
    
    df1= pd.DataFrame(ret[d1][:])
    df1.columns=['涨跌幅']
    df1['流通市值']=LTSZ[d0]
    
    df1= df1.dropna()
    
    Y=df1['涨跌幅']
    X=df1[df1.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    result = np.array(est.params[['流通市值']])
    ratio = abs(result)/sum(abs(result))
    
    data_LTSZ[d0]=LTSZ[d0]*ratio[0]
    
data_LTSZ.dropna()[:3]

,2012-12-31,2013-01-31,2013-02-28,2013-03-29,2013-04-26,2013-05-31,2013-06-28,2013-07-31,2013-08-30,2013-09-30,...,2019-02-28,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,4.974784e+10,6.552305e+10,7.142323e+10,6.247980e+10,5.807020e+10,6.518146e+10,4.953668e+10,4.764862e+10,5.271657e+10,5.902666e+10,...,2.122243e+11,2.201226e+11,2.378079e+11,2.091336e+11,2.366060e+11,2.426156e+11,2.431307e+11,3.025357e+11,3.155375e+11,2.967140e+11
000002.SZ,9.778141e+10,1.160430e+11,1.159463e+11,1.039652e+11,1.066976e+11,1.158010e+11,9.521204e+10,9.210544e+10,9.152494e+10,8.833221e+10,...,2.719276e+11,2.984500e+11,2.805741e+11,2.593950e+11,2.701789e+11,2.794083e+11,2.506514e+11,2.516229e+11,2.577435e+11,2.691102e+11
000004.SZ,6.861013e+08,7.817193e+08,8.429484e+08,8.001719e+08,8.924349e+08,9.838591e+08,8.387546e+08,8.463034e+08,1.019926e+09,1.067735e+09,...,1.613059e+09,1.797171e+09,2.096562e+09,1.783073e+09,1.759851e+09,1.528185e+09,1.638875e+09,1.552966e+09,1.502577e+09,1.644657e+09


In [ ]:
TotalAssets= pd.read_excel(r'D:\data\月度数据2_2.0.xlsx', sheet_name = '资产总计', header =1, index_col = 0)
Liabilities= pd.read_excel(r'D:\data\月度数据2_2.0.xlsx', sheet_name = '非流动负债合计', header =1, index_col = 0)
TotalValue= pd.read_excel(r'D:\data\月度数据2_2.0.xlsx', sheet_name = '总市值', header =1, index_col = 0)

In [ ]:
#涨跌幅数据
ret1 = pd.read_excel(r'D:\data\月度数据2_2.0.xlsx', sheet_name = '涨跌幅', header =1, index_col = 0)
ret1

,2012-03-30,2012-04-27,2012-05-31,2012-06-29,2012-07-31,2012-08-31,2012-09-28,2012-10-31,2012-11-30,2012-12-31,...,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29,2019-12-31
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,-8.128655,5.410567,-5.253623,-3.377948,-0.395778,-6.026490,-7.470049,0.975900,-2.735562,25.156250,...,3.721683,8.034321,-12.057762,14.328093,2.539913,0.212314,10.098870,4.297627,-5.965560,7.586658
000002.SZ,0.000000,8.454106,3.006682,-3.675676,5.091124,-13.109426,5.112219,-1.304864,5.168269,15.657143,...,9.753483,-5.989583,-7.548476,4.157303,3.416037,-6.680365,0.387597,2.432432,4.410102,16.173285
000004.SZ,-13.785790,4.920049,10.082063,-9.797657,-5.785124,-0.501253,2.267003,1.354680,-12.029162,12.983425,...,11.413882,16.658976,-14.952532,-1.302326,-12.818096,7.243243,-5.241935,-3.244681,9.455745,13.460573
000005.SZ,NaN,2.849741,-11.083123,-14.730878,-9.302326,1.831502,2.517986,1.754386,-13.103448,17.857143,...,9.593023,-12.466844,-8.181818,0.330033,-3.947368,6.164384,2.580645,-5.031447,-2.980132,5.460751
000006.SZ,4.554080,15.245009,-3.622047,-6.699346,-6.064823,-2.195122,0.000000,3.491272,-6.987952,28.238342,...,10.655738,-9.629630,-10.000000,2.751396,-2.935780,0.189036,-5.094340,3.976143,-2.103250,4.687500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688699.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688777.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688788.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
date1 = TotalAssets.columns

BM = pd.DataFrame()

for i in range (len(date1)):
    BM[date1[i]] = (TotalAssets[date1[i]]-Liabilities[date1[i]]
                                 )/TotalValue[date1[i]]
BM


,2012-03-30,2012-04-27,2012-05-31,2012-06-29,2012-07-31,2012-08-31,2012-09-28,2012-10-31,2012-11-30,2012-12-31,...,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29,2019-12-31
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,0.936546,0.930633,0.982236,1.016576,1.031799,1.097967,1.186608,1.212131,1.246223,1.033173,...,1.090482,1.055203,1.199882,1.060563,1.057642,1.055402,0.952187,0.912952,0.970870,0.980439
000002.SZ,0.745082,0.700049,0.679615,0.705549,0.688116,0.791934,0.753417,0.783130,0.744645,0.738155,...,0.694795,0.723606,0.782687,0.751447,0.715176,0.797227,0.836456,0.816593,0.782101,0.743956
000004.SZ,0.202729,0.194845,0.177000,0.196225,0.208602,0.209653,0.205005,0.205356,0.233436,0.215533,...,0.100741,0.063548,0.074720,0.075706,0.081148,0.075667,0.074480,0.076978,0.070328,0.740344
000005.SZ,0.186986,0.178437,0.200679,0.235348,0.252662,0.248118,0.242024,0.243837,0.280606,0.251181,...,0.395751,0.446974,0.486803,0.485202,0.532621,0.501695,0.484120,0.509769,0.525427,0.532666
000006.SZ,0.530134,0.478836,0.496831,0.532506,0.609397,0.623074,0.623074,0.604747,0.650181,0.557639,...,0.700813,0.805082,0.894535,0.901101,0.905044,0.903336,0.987997,0.950215,0.970630,0.990914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688699.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688777.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688788.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def find_tvalue(x,index):
    test=pd.DataFrame(ret1[date1[x+1]][:])
    test.columns = ['涨跌幅']
    
    test['BM']=BM[date1[x]][:]
    test['行业类别']= df_stock['行业类别'].values
    
    #去极值
    test['涨跌幅']= mad(test['涨跌幅'],5)
    test['BM']= mad(test['BM'],5)
    
    #标准化
    test['涨跌幅']=standard(test['涨跌幅'])
    test['BM']=standard(test['BM'])
    
    #去缺失值
    test= test.dropna()
    
    #行业dummy
    test=pd.get_dummies(test)
    
    #回归
    Y=test['涨跌幅']
    X=test[test.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    return est.tvalues[index],est.params[index]
find_tvalue(0,'BM')[0]

7.695728957149816

In [ ]:
index = 'BM'
result_t = []
result_r=[]
for i in range(0,len(date1)-1):
    result_t.append(find_tvalue(i,index)[0])
    result_r.append(find_tvalue(i,index)[1])
result_t[:3]

[7.695728957149816, -1.3710656598968716, -2.8016987912693403]

In [ ]:
def score_1(index):
    result_t=[]
    result_r=[]
    for i in range(len(date1)-1):
        result_t.append(find_tvalue(i,index)[0])
        result_r.append(find_tvalue(i,index)[1])
    score1=np.mean([abs(x) for x in result_t])
    print('T值序列的绝对值平均值:',score1)
    score2=np.mean([abs(x)>2 for x in result_t])
    print('T值序列的绝对值大于2的比例:',score2)
    score3=stats.ttest_1samp(result_r,0)[0]
    print('收益率序列的T值',score3)
    return score1, score2, score3

In [ ]:
score_1('BM')

T值序列的绝对值平均值: 4.988549006554826
T值序列的绝对值大于2的比例: 0.6774193548387096
收益率序列的T值 3.067896663059994


(4.988549006554826, 0.6774193548387096, 3.067896663059994)

In [ ]:
data_BM = pd.DataFrame()

for i in range(11, len(date)-1):
    d0= date[i]
    d1= date[i+1]
    
    df1= pd.DataFrame(ret[d1][:])
    df1.columns=['涨跌幅']
    df1['BM']=BM[d0]
    
    df1= df1.dropna()
    
    Y=df1['涨跌幅']
    X=df1[df1.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    result = np.array(est.params[['BM']])
    ratio = abs(result)/sum(abs(result))
    
    data_BM[d0]=BM[d0]*ratio[0]
    
data_BM.dropna()[:3]

,2012-12-31,2013-01-31,2013-02-28,2013-03-29,2013-04-26,2013-05-31,2013-06-28,2013-07-31,2013-08-30,2013-09-30,...,2019-02-28,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,1.033173,0.784428,0.719627,0.822636,0.926439,0.825365,1.086033,1.164469,1.052521,0.974620,...,1.131066,1.090482,1.055203,1.199882,1.060563,1.057642,1.055402,0.952187,0.912952,0.970870
000002.SZ,0.738155,0.621992,0.622511,0.694250,0.697061,0.641737,0.780508,0.839913,0.845241,0.938454,...,0.762562,0.694795,0.723606,0.782687,0.751447,0.715176,0.797227,0.836456,0.816593,0.782101
000004.SZ,0.215533,0.189169,0.175429,0.184807,0.167831,0.152235,0.178572,0.178530,0.148139,0.144626,...,0.112240,0.100741,0.063548,0.074720,0.075706,0.081148,0.075667,0.074480,0.076978,0.070328


In [ ]:
EPS_TTM= pd.read_excel(r'D:\data\月度数据2_2.0.xlsx', sheet_name = '每股收益EPS_TTM', header =1, index_col = 0)

In [ ]:
def find_tvalue(x,index):
    test=pd.DataFrame(ret1[date1[x+1]][:])
    test.columns = ['涨跌幅']
    
    test['EPS_TTM']=EPS_TTM[date1[x]][:]
    test['行业类别']= df_stock['行业类别'].values
    
    #去极值
    test['涨跌幅']= mad(test['涨跌幅'],5)
    test['EPS_TTM']= mad(test['EPS_TTM'],5)
    
    #标准化
    test['涨跌幅']=standard(test['涨跌幅'])
    test['EPS_TTM']=standard(test['EPS_TTM'])
    
    #去缺失值
    test= test.dropna()
    
    #行业dummy
    test=pd.get_dummies(test)
    
    #回归
    Y=test['涨跌幅']
    X=test[test.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    return est.tvalues[index],est.params[index]
find_tvalue(0,'EPS_TTM')[0]

-7.468199140855538

In [ ]:
index = 'EPS_TTM'
result_t = []
result_r=[]
for i in range(0,len(date1)-1):
    result_t.append(find_tvalue(i,index)[0])
    result_r.append(find_tvalue(i,index)[1])
result_t[:3]

[-7.468199140855538, 5.609224530371589, 7.57049602364373]

In [ ]:
score_1('EPS_TTM')

T值序列的绝对值平均值: 4.961073697185932
T值序列的绝对值大于2的比例: 0.7741935483870968
收益率序列的T值 2.2140195722444167


(4.961073697185932, 0.7741935483870968, 2.2140195722444167)

In [ ]:
data_EPS_TTM = pd.DataFrame()

for i in range(11, len(date)-1):
    d0= date[i]
    d1= date[i+1]
    
    df1= pd.DataFrame(ret[d1][:])
    df1.columns=['涨跌幅']
    df1['EPS_TTM']=EPS_TTM[d0]
    
    df1= df1.dropna()
    
    Y=df1['涨跌幅']
    X=df1[df1.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    result = np.array(est.params[['EPS_TTM']])
    ratio = abs(result)/sum(abs(result))
    
    data_EPS_TTM[d0]=EPS_TTM[d0]*ratio[0]
    
data_EPS_TTM.dropna()[:3]

,2012-12-31,2013-01-31,2013-02-28,2013-03-29,2013-04-26,2013-05-31,2013-06-28,2013-07-31,2013-08-30,2013-09-30,...,2019-02-28,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,2.504092,2.504092,2.504092,2.616003,2.647233,2.647233,1.654520,1.654520,1.728935,1.728935,...,1.445393,1.445393,1.494955,1.494955,1.494955,1.494955,1.563678,1.383547,1.441983,1.441983
000002.SZ,1.011395,1.011395,1.141478,1.141478,1.160119,1.160031,1.160031,1.159385,1.215066,1.215066,...,2.803238,3.059352,3.008138,3.008138,3.008138,3.008138,3.228650,3.228650,3.364746,3.364746
000004.SZ,0.024377,0.024377,0.024377,0.024377,0.033393,0.033393,0.033393,0.033393,0.033906,0.033906,...,0.135862,0.135862,-0.202214,-0.202214,-0.202214,-0.202214,-0.202214,-0.287859,-0.328903,-0.328903


In [ ]:
debtRatio = pd.DataFrame()
for i in range(0, len(date1)):
    debtRatio[date1[i]] = Liabilities[date1[i]]/TotalAssets[date1[i]]
debtRatio

,2012-03-30,2012-04-27,2012-05-31,2012-06-29,2012-07-31,2012-08-31,2012-09-28,2012-10-31,2012-11-30,2012-12-31,...,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29,2019-12-31
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,0.940087,0.942283,0.942283,0.942283,0.946450,0.946450,0.946450,0.944688,0.944688,0.947216,...,0.929783,0.928916,0.928916,0.928916,0.928538,0.928538,0.922304,0.922304,0.922304,0.920544
000002.SZ,0.770997,0.777343,0.777343,0.777343,0.788639,0.788639,0.788639,0.794002,0.794002,0.783163,...,0.845856,0.847734,0.847734,0.847734,0.852594,0.852594,0.850587,0.850587,0.850587,0.843590
000004.SZ,0.294935,0.292413,0.292413,0.292413,0.306461,0.306461,0.306461,0.237050,0.237050,0.232528,...,0.477966,0.163786,0.163786,0.163786,0.149427,0.149427,0.095336,0.095336,0.095336,0.060243
000005.SZ,0.476484,0.483775,0.483775,0.483775,0.488110,0.488110,0.488110,0.477673,0.477673,0.483142,...,0.494040,0.471118,0.471118,0.471118,0.436745,0.436745,0.468174,0.468174,0.468174,0.425337
000006.SZ,0.652855,0.638762,0.638762,0.638762,0.616302,0.616302,0.616302,0.641087,0.641087,0.620165,...,0.528236,0.527107,0.527107,0.527107,0.584785,0.584785,0.566858,0.566858,0.566858,0.544612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688699.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.258038,0.258038,0.258038,0.258038,0.258038,0.258038,NaN,NaN,NaN,0.250581
688777.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.679574,0.679574,0.679574,0.679574,0.679574,0.679574,0.639181,0.639181,0.639181,0.635090
688788.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.426507,0.426507,0.426507,0.426507,NaN,NaN,NaN,NaN,NaN,0.351647


In [ ]:
def find_tvalue(x,index):
    test=pd.DataFrame(ret1[date1[x+1]][:])
    test.columns = ['涨跌幅']
    
    test['debtRatio']=debtRatio[date1[x]][:]
    test['行业类别']= df_stock['行业类别'].values
    
    #去极值
    test['涨跌幅']= mad(test['涨跌幅'],5)
    test['debtRatio']= mad(test['debtRatio'],5)
    
    #标准化
    test['涨跌幅']=standard(test['涨跌幅'])
    test['debtRatio']=standard(test['debtRatio'])
    
    #去缺失值
    test= test.dropna()
    
    #行业dummy
    test=pd.get_dummies(test)
    
    #回归
    Y=test['涨跌幅']
    X=test[test.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    return est.tvalues[index],est.params[index]
find_tvalue(0,'debtRatio')[0]

5.565457870195418

In [ ]:
index = 'debtRatio'
result_t = []
result_r=[]
for i in range(0,len(date1)-1):
    result_t.append(find_tvalue(i,index)[0])
    result_r.append(find_tvalue(i,index)[1])
result_t[:3]

[5.565457870195418, -4.2535216857125215, -7.233668262410626]

In [ ]:
score_1('debtRatio')

T值序列的绝对值平均值: 3.6692438966683816
T值序列的绝对值大于2的比例: 0.6774193548387096
收益率序列的T值 -0.8109876171025238


(3.6692438966683816, 0.6774193548387096, -0.8109876171025238)

In [ ]:
data_debtRatio = pd.DataFrame()

for i in range(11, len(date)-1):
    d0= date[i]
    d1= date[i+1]
    
    df1= pd.DataFrame(ret[d1][:])
    df1.columns=['涨跌幅']
    df1['debtRatio']=debtRatio[d0]
    
    df1= df1.dropna()
    
    Y=df1['涨跌幅']
    X=df1[df1.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    result = np.array(est.params[['debtRatio']])
    ratio = abs(result)/sum(abs(result))
    
    data_debtRatio[d0]=debtRatio[d0]*ratio[0]
    
data_debtRatio.dropna()[:3]

,2012-12-31,2013-01-31,2013-02-28,2013-03-29,2013-04-26,2013-05-31,2013-06-28,2013-07-31,2013-08-30,2013-09-30,...,2019-02-28,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,0.947216,0.947216,0.947216,0.947216,0.950996,0.950996,0.950996,0.949895,0.949895,0.948851,...,0.929783,0.929783,0.928916,0.928916,0.928916,0.928538,0.928538,0.922304,0.922304,0.922304
000002.SZ,0.783163,0.783163,0.783163,0.783163,0.797493,0.797493,0.797493,0.796259,0.796259,0.795362,...,0.845856,0.845856,0.847734,0.847734,0.847734,0.852594,0.852594,0.850587,0.850587,0.850587
000004.SZ,0.232528,0.232528,0.232528,0.232528,0.178113,0.178113,0.178113,0.236968,0.236968,0.286237,...,0.477966,0.477966,0.163786,0.163786,0.163786,0.149427,0.149427,0.095336,0.095336,0.095336


In [ ]:
BookValue = pd.DataFrame()
for i in range(0, len(date1)):
    BookValue[date1[i]] = TotalAssets[date1[i]]-Liabilities[date1[i]]
BookValue

,2012-03-30,2012-04-27,2012-05-31,2012-06-29,2012-07-31,2012-08-31,2012-09-28,2012-10-31,2012-11-30,2012-12-31,...,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29,2019-12-31
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,7.538058e+10,7.895742e+10,7.895742e+10,7.895742e+10,7.982261e+10,7.982261e+10,7.982261e+10,8.172588e+10,8.172588e+10,8.479878e+10,...,2.400420e+11,2.509380e+11,2.509380e+11,2.509380e+11,2.566030e+11,2.566030e+11,2.880730e+11,2.880730e+11,2.880730e+11,3.129830e+11
000002.SZ,6.783254e+10,6.912071e+10,6.912071e+10,6.912071e+10,6.983397e+10,6.983397e+10,6.983397e+10,7.164086e+10,7.164086e+10,8.213819e+10,...,2.356207e+11,2.361894e+11,2.361894e+11,2.361894e+11,2.324675e+11,2.324675e+11,2.448519e+11,2.448519e+11,2.448519e+11,2.705791e+11
000004.SZ,1.384093e+08,1.395716e+08,1.395716e+08,1.395716e+08,1.397913e+08,1.397913e+08,1.397913e+08,1.419271e+08,1.419271e+08,1.480558e+08,...,1.833266e+08,1.349077e+08,1.349077e+08,1.349077e+08,1.260689e+08,1.260689e+08,1.175867e+08,1.175867e+08,1.175867e+08,1.404457e+09
000005.SZ,6.599366e+08,6.477110e+08,6.477110e+08,6.477110e+08,6.306784e+08,6.306784e+08,6.306784e+08,6.465490e+08,6.465490e+08,6.820994e+08,...,1.579316e+09,1.561356e+09,1.561356e+09,1.561356e+09,1.646294e+09,1.646294e+09,1.629619e+09,1.629619e+09,1.629619e+09,1.742286e+09
000006.SZ,2.888930e+09,3.007182e+09,3.007182e+09,3.007182e+09,3.212379e+09,3.212379e+09,3.212379e+09,3.226743e+09,3.226743e+09,3.548962e+09,...,6.386137e+09,6.629824e+09,6.629824e+09,6.629824e+09,6.463349e+09,6.463349e+09,6.708967e+09,6.708967e+09,6.708967e+09,7.170229e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688699.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.677252e+08,2.677252e+08,2.677252e+08,2.677252e+08,2.677252e+08,2.677252e+08,NaN,NaN,NaN,3.353179e+08
688777.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.182839e+09,1.182839e+09,1.182839e+09,1.182839e+09,1.182839e+09,1.182839e+09,1.680906e+09,1.680906e+09,1.680906e+09,1.850151e+09
688788.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.657128e+08,3.657128e+08,3.657128e+08,3.657128e+08,NaN,NaN,NaN,NaN,NaN,7.575247e+08


In [ ]:
def find_tvalue(x,index):
    test=pd.DataFrame(ret1[date1[x+1]][:])
    test.columns = ['涨跌幅']
    
    test['BookValue']=BookValue[date1[x]][:]
    test['行业类别']= df_stock['行业类别'].values
    
    #去极值
    test['涨跌幅']= mad(test['涨跌幅'],5)
    test['BookValue']= mad(test['BookValue'],5)
    
    #标准化
    test['涨跌幅']=standard(test['涨跌幅'])
    test['BookValue']=standard(test['BookValue'])
    
    #去缺失值
    test= test.dropna()
    
    #行业dummy
    test=pd.get_dummies(test)
    
    #回归
    Y=test['涨跌幅']
    X=test[test.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    return est.tvalues[index],est.params[index]
find_tvalue(0,'BookValue')[0]

1.837308808187135

In [ ]:
index = 'BookValue'
result_t = []
result_r=[]
for i in range(0,len(date1)-1):
    result_t.append(find_tvalue(i,index)[0])
    result_r.append(find_tvalue(i,index)[1])
result_t[:3]

[1.837308808187135, -1.6861599800207507, -5.6567694231356755]

In [ ]:
score_1('BookValue')

T值序列的绝对值平均值: 6.361114333071535
T值序列的绝对值大于2的比例: 0.7634408602150538
收益率序列的T值 -1.7696209517872339


(6.361114333071535, 0.7634408602150538, -1.7696209517872339)

In [ ]:
data_BookValue = pd.DataFrame()

for i in range(11,len(date)-1):
    d0= date[i]
    d1= date[i+1]
    
    df1= pd.DataFrame(ret[d1][:])
    df1.columns=['涨跌幅']
    df1['BookValue']=BookValue[d0]
    
    df1= df1.dropna()
    
    Y=df1['涨跌幅']
    X=df1[df1.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    result = np.array(est.params[['BookValue']])
    ratio = abs(result)/sum(abs(result))
    
    data_BookValue[d0]=BookValue[d0]*ratio[0]
    
data_BookValue.dropna()[:3]

,2012-12-31,2013-01-31,2013-02-28,2013-03-29,2013-04-26,2013-05-31,2013-06-28,2013-07-31,2013-08-30,2013-09-30,...,2019-02-28,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,8.479878e+10,8.479878e+10,8.479878e+10,8.479878e+10,8.875900e+10,8.875900e+10,8.875900e+10,9.154200e+10,9.154200e+10,9.491300e+10,...,2.400420e+11,2.400420e+11,2.509380e+11,2.509380e+11,2.509380e+11,2.566030e+11,2.566030e+11,2.880730e+11,2.880730e+11,2.880730e+11
000002.SZ,8.213819e+10,8.213819e+10,8.213819e+10,8.213819e+10,8.462643e+10,8.462643e+10,8.462643e+10,8.806548e+10,8.806548e+10,9.436654e+10,...,2.356207e+11,2.356207e+11,2.361894e+11,2.361894e+11,2.361894e+11,2.324675e+11,2.324675e+11,2.448519e+11,2.448519e+11,2.448519e+11
000004.SZ,1.480558e+08,1.480558e+08,1.480558e+08,1.480558e+08,1.499590e+08,1.499590e+08,1.499590e+08,1.512726e+08,1.512726e+08,1.546085e+08,...,1.833266e+08,1.833266e+08,1.349077e+08,1.349077e+08,1.349077e+08,1.260689e+08,1.260689e+08,1.175867e+08,1.175867e+08,1.175867e+08


In [ ]:
ValueLeverage = pd.DataFrame()
for i in range(0, len(date1)):
    ValueLeverage[date1[i]] = (TotalValue[date1[i]]+Liabilities[date1[i]])/TotalValue[date1[i]]
ValueLeverage

,2012-03-30,2012-04-27,2012-05-31,2012-06-29,2012-07-31,2012-08-31,2012-09-28,2012-10-31,2012-11-30,2012-12-31,...,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29,2019-12-31
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,15.695343,16.193386,17.035849,17.596469,19.236207,20.405689,21.972333,21.702117,22.284364,19.540580,...,15.439772,14.789325,16.679979,14.859372,14.742444,14.713328,12.303066,11.837319,12.524840,12.358948
000002.SZ,3.508513,3.444026,3.372687,3.463227,3.567524,3.954894,3.811180,4.018510,3.870172,3.666038,...,4.812657,5.028656,5.357587,5.183660,5.136566,5.611149,5.761838,5.648760,5.452404,5.012479
000004.SZ,1.084803,1.080520,1.073146,1.081091,1.092177,1.092641,1.090588,1.063805,1.072529,1.065302,...,1.092237,1.012447,1.014635,1.014828,1.014256,1.013293,1.007849,1.008112,1.007411,1.047460
000005.SZ,1.170188,1.167221,1.188064,1.220554,1.240924,1.236591,1.230780,1.222991,1.256616,1.234795,...,1.386427,1.398155,1.433634,1.432208,1.412991,1.389011,1.426178,1.448757,1.462542,1.394254
000006.SZ,1.996993,1.846704,1.878525,1.941606,1.978821,2.000790,2.000790,2.080193,2.161347,1.910470,...,1.784704,1.897379,1.997088,2.004406,2.274653,2.272248,2.293005,2.243559,2.270276,2.185064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688699.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688777.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688788.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def find_tvalue(x,index):
    test=pd.DataFrame(ret1[date1[x+1]][:])
    test.columns = ['涨跌幅']
    
    test['ValueLeverage']=ValueLeverage[date1[x]][:]
    test['行业类别']= df_stock['行业类别'].values
    
    #去极值
    test['涨跌幅']= mad(test['涨跌幅'],5)
    test['ValueLeverage']= mad(test['ValueLeverage'],5)
    
    #标准化
    test['涨跌幅']=standard(test['涨跌幅'])
    test['ValueLeverage']=standard(test['ValueLeverage'])
    
    #去缺失值
    test= test.dropna()
    
    #行业dummy
    test=pd.get_dummies(test)
    
    #回归
    Y=test['涨跌幅']
    X=test[test.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    return est.tvalues[index],est.params[index]
find_tvalue(0,'ValueLeverage')[0]

9.114423447625745

In [ ]:
index = 'ValueLeverage'
result_t = []
result_r=[]
for i in range(0,len(date1)-1):
    result_t.append(find_tvalue(i,index)[0])
    result_r.append(find_tvalue(i,index)[1])
result_t[:3]

[9.114423447625745, -3.9745788018955857, -7.6811436281873835]

In [ ]:
score_1('ValueLeverage')

T值序列的绝对值平均值: 4.927692873327779
T值序列的绝对值大于2的比例: 0.7419354838709677
收益率序列的T值 0.8867632648252975


(4.927692873327779, 0.7419354838709677, 0.8867632648252975)

In [ ]:
data_ValueLeverage = pd.DataFrame()

for i in range(11,len(date)-1):
    d0= date[i]
    d1= date[i+1]
    
    df1= pd.DataFrame(ret[d1][:])
    df1.columns=['涨跌幅']
    df1['ValueLeverage']=ValueLeverage[d0]
    
    df1= df1.dropna()
    
    Y=df1['涨跌幅']
    X=df1[df1.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    result = np.array(est.params[['ValueLeverage']])
    ratio = abs(result)/sum(abs(result))
    
    data_ValueLeverage[d0]=ValueLeverage[d0]*ratio[0]
    
data_ValueLeverage.dropna()[:3]

,2012-12-31,2013-01-31,2013-02-28,2013-03-29,2013-04-26,2013-05-31,2013-06-28,2013-07-31,2013-08-30,2013-09-30,...,2019-02-28,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,19.540580,15.076781,13.913917,15.762430,18.978814,17.017333,22.075967,23.076029,20.953734,19.079914,...,15.977174,15.439772,14.789325,16.679979,14.859372,14.742444,14.713328,12.303066,11.837319,12.524840
000002.SZ,3.666038,3.246487,3.248359,3.507464,3.745102,3.527227,4.073724,4.282540,4.303359,4.647458,...,5.184524,4.812657,5.028656,5.357587,5.183660,5.136566,5.611149,5.761838,5.648760,5.452404
000004.SZ,1.065302,1.057314,1.053151,1.055993,1.036371,1.032991,1.038699,1.055445,1.046006,1.057999,...,1.102765,1.092237,1.012447,1.014635,1.014828,1.014256,1.013293,1.007849,1.008112,1.007411


In [ ]:
NetAssets = pd.read_excel(r'D:\data\月度数据2_2.0.xlsx', sheet_name = '每股净资产', header =1, index_col = 0)

In [ ]:
NetAssetsRate = pd.DataFrame()
for i in range(1, len(date1)):
    NetAssetsRate[date1[i]] = (NetAssets[date1[i]]-NetAssets[date1[i-1]])/NetAssets[date1[i-1]]
NetAssetsRate 

,2012-04-27,2012-05-31,2012-06-29,2012-07-31,2012-08-31,2012-09-28,2012-10-31,2012-11-30,2012-12-31,2013-01-31,...,2019-03-29,2019-04-30,2019-05-31,2019-06-28,2019-07-31,2019-08-30,2019-09-30,2019-10-31,2019-11-29,2019-12-31
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,0.047803,0.000000,0.0,0.000000,0.039150,-0.006418,0.030458,0.0,0.000000,0.000000,...,0.000005,0.032582,-0.010955,0.0,0.000000,0.036280,-0.115197,0.151100,0.0,0.0
000002.SZ,0.026252,-0.026295,0.0,0.000000,0.041015,0.000000,0.022177,0.0,0.000022,0.000000,...,0.112590,-0.012062,0.000000,0.0,-0.074972,0.104826,0.000000,0.038859,0.0,0.0
000004.SZ,0.031058,0.000000,0.0,0.000000,-0.010570,0.000000,0.004940,0.0,0.000000,0.000000,...,0.000000,-0.098255,0.000000,0.0,0.000000,0.000000,-0.063436,-0.074754,0.0,0.0
000005.SZ,-0.077073,0.000000,0.0,0.000000,-0.026297,0.000000,0.025164,0.0,0.000000,0.000000,...,0.000000,0.107194,0.000000,0.0,0.000000,0.056340,0.000000,-0.011377,0.0,0.0
000006.SZ,0.041351,-0.013587,0.0,-0.230769,0.088187,0.000000,0.004206,0.0,0.000000,0.103257,...,0.000000,0.016095,-0.041729,0.0,0.000000,0.015306,0.000000,0.040475,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688699.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
688777.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688788.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def find_tvalue(x,index):
    test=pd.DataFrame(ret1[date1[x+2]][:])
    test.columns = ['涨跌幅']
    
    test['NetAssetsRate']=NetAssetsRate[date1[x+1]][:]
    test['行业类别']= df_stock['行业类别'].values
    
    #去极值
    test['涨跌幅']= mad(test['涨跌幅'],5)
    test['NetAssetsRate']= mad(test['NetAssetsRate'],5)
    
    #标准化
    test['涨跌幅']=standard(test['涨跌幅'])
    test['NetAssetsRate']=standard(test['NetAssetsRate'])
    
    #去缺失值
    test= test.dropna()
    
    #行业dummy
    test=pd.get_dummies(test)
    
    #回归
    Y=test['涨跌幅']
    X=test[test.columns[1:]]
    est=sm.OLS(Y,sm.add_constant(X)).fit()
    
    return est.tvalues[index],est.params[index]
find_tvalue(0,'NetAssetsRate')[0]

2.8227609894028265

In [ ]:
date2 = NetAssetsRate.columns

In [ ]:
index = 'NetAssetsRate'
result_t = []
result_r=[]
for i in range(0,len(date2)-1):
    result_t.append(find_tvalue(i,index)[0])
    result_r.append(find_tvalue(i,index)[1])
result_t[:3]